In [ ]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
%run "/Formula1/includes/configuration"

In [ ]:
dbutils.fs.ls(raw_folder_path)

Out[15]: [FileInfo(path='dbfs:/mnt/formularacedata/raw/circuits.csv', name='circuits.csv', size=10044, modificationTime=1686570285000),
 FileInfo(path='dbfs:/mnt/formularacedata/raw/constructors/', name='constructors/', size=0, modificationTime=1686899480000),
 FileInfo(path='dbfs:/mnt/formularacedata/raw/constructors.json', name='constructors.json', size=30415, modificationTime=1686570285000),
 FileInfo(path='dbfs:/mnt/formularacedata/raw/drivers.json', name='drivers.json', size=180812, modificationTime=1686570285000),
 FileInfo(path='dbfs:/mnt/formularacedata/raw/lap_times/', name='lap_times/', size=0, modificationTime=1686570362000),
 FileInfo(path='dbfs:/mnt/formularacedata/raw/pit_stops.json', name='pit_stops.json', size=1369387, modificationTime=1686925342000),
 FileInfo(path='dbfs:/mnt/formularacedata/raw/qualifying/', name='qualifying/', size=0, modificationTime=1686570383000),
 FileInfo(path='dbfs:/mnt/formularacedata/raw/races.csv', name='races.csv', size=116847, modification

In [ ]:
dbutils.fs.head('/mnt/formularacedata/raw/qualifying/qualifying_split_1.json')

[Truncated to first 65536 bytes]
Out[16]: '[\n  {\n    "qualifyId": 1,\n    "raceId": 18,\n    "driverId": 1,\n    "constructorId": 1,\n    "number": 22,\n    "position": 1,\n    "q1": "1:26.572",\n    "q2": "1:25.187",\n    "q3": "1:26.714"\n  },\n  {\n    "qualifyId": 2,\n    "raceId": 18,\n    "driverId": 9,\n    "constructorId": 2,\n    "number": 4,\n    "position": 2,\n    "q1": "1:26.103",\n    "q2": "1:25.315",\n    "q3": "1:26.869"\n  },\n  {\n    "qualifyId": 3,\n    "raceId": 18,\n    "driverId": 5,\n    "constructorId": 1,\n    "number": 23,\n    "position": 3,\n    "q1": "1:25.664",\n    "q2": "1:25.452",\n    "q3": "1:27.079"\n  },\n  {\n    "qualifyId": 4,\n    "raceId": 18,\n    "driverId": 13,\n    "constructorId": 6,\n    "number": 2,\n    "position": 4,\n    "q1": "1:25.994",\n    "q2": "1:25.691",\n    "q3": "1:27.178"\n  },\n  {\n    "qualifyId": 5,\n    "raceId": 18,\n    "driverId": 2,\n    "constructorId": 2,\n    "number": 3,\n    "position": 5,\n    "q1": "1:25

In [ ]:
schema = 'qualifyId int, raceId int, driverId int, constructorId int, number int, q1 String, q2 STring, q3 String'
file_name = f'{raw_folder_path}/qualifying/'

df = spark.read.format('json').option('multiLine',True).schema(schema).load(file_name)
df.show(3)

+---------+------+--------+-------------+------+--------+--------+--------+
|qualifyId|raceId|driverId|constructorId|number|      q1|      q2|      q3|
+---------+------+--------+-------------+------+--------+--------+--------+
|        1|    18|       1|            1|    22|1:26.572|1:25.187|1:26.714|
|        2|    18|       9|            2|     4|1:26.103|1:25.315|1:26.869|
|        3|    18|       5|            1|    23|1:25.664|1:25.452|1:27.079|
+---------+------+--------+-------------+------+--------+--------+--------+
only showing top 3 rows



In [ ]:
from pyspark.sql.functions import current_timestamp

df_new = df.withColumnRenamed('qualifyId','qualify_id')\
            .withColumnRenamed('raceId','race_id') \
            .withColumnRenamed('driverId','driver_id') \
            .withColumnRenamed('constructorId', 'constructor_id')  \
            .withColumn('ingestion_date',current_timestamp())

df_new.show(4)
            

+----------+-------+---------+--------------+------+--------+--------+--------+--------------------+
|qualify_id|race_id|driver_id|constructor_id|number|      q1|      q2|      q3|      ingestion_date|
+----------+-------+---------+--------------+------+--------+--------+--------+--------------------+
|         1|     18|        1|             1|    22|1:26.572|1:25.187|1:26.714|2023-06-16 19:12:...|
|         2|     18|        9|             2|     4|1:26.103|1:25.315|1:26.869|2023-06-16 19:12:...|
|         3|     18|        5|             1|    23|1:25.664|1:25.452|1:27.079|2023-06-16 19:12:...|
|         4|     18|       13|             6|     2|1:25.994|1:25.691|1:27.178|2023-06-16 19:12:...|
+----------+-------+---------+--------------+------+--------+--------+--------+--------------------+
only showing top 4 rows



In [ ]:
df_new.count()

Out[19]: 8694

In [ ]:
dest_path = f'{processed_folder_path}/qualifying'
df_new.write.mode('overwrite').format('parquet').option('path',dest_path).save()

In [ ]:
spark.read.format('parquet').load(f'{dest_path}/*').show(4)

+----------+-------+---------+--------------+------+--------+--------+--------+--------------------+
|qualify_id|race_id|driver_id|constructor_id|number|      q1|      q2|      q3|      ingestion_date|
+----------+-------+---------+--------------+------+--------+--------+--------+--------------------+
|         1|     18|        1|             1|    22|1:26.572|1:25.187|1:26.714|2023-06-16 19:12:...|
|         2|     18|        9|             2|     4|1:26.103|1:25.315|1:26.869|2023-06-16 19:12:...|
|         3|     18|        5|             1|    23|1:25.664|1:25.452|1:27.079|2023-06-16 19:12:...|
|         4|     18|       13|             6|     2|1:25.994|1:25.691|1:27.178|2023-06-16 19:12:...|
+----------+-------+---------+--------------+------+--------+--------+--------+--------------------+
only showing top 4 rows



In [ ]:
dbutils.notebook.exit("success")